In [1]:
# import required libraries
import requests
import pandas as pd

# alpha_vantage library does not yet support ETF Profile & Holdings API
# so we need to use the following API to get the ETF profile and holdings
# https://www.alphavantage.co/documentation/etf-profile-and-holdings


In [3]:
import yaml

def get_alpha_vantage_api_key(config_file='config.yaml'):
    """
    Reads the Alpha Vantage API key from a YAML configuration file.

    Args:
        config_file (str): Path to the YAML configuration file.

    Returns:
        str: The Alpha Vantage API key.

    Raises:
        KeyError: If the key is not found in the YAML file.
        FileNotFoundError: If the config file does not exist.
    """
    try:
        with open(config_file, 'r') as file:
            config = yaml.safe_load(file)
            return config['alpha_vantage']['api_key']
    except FileNotFoundError:
        raise FileNotFoundError(f"Configuration file {config_file} not found.")
    except KeyError:
        raise KeyError("API key not found in the configuration file. Ensure the key exists under 'alpha_vantage.api_key'.")



In [6]:
api_key = None
try:
    api_key = get_alpha_vantage_api_key(config_file='../config.yml')
except Exception as e:
    print(f"Error: {e}")

In [13]:
import requests
import pandas as pd

def get_etf_holdings(api_key, etf_symbol):
    """
    Retrieves ETF holdings from Alpha Vantage.

    Args:
        api_key (str): Your Alpha Vantage API key.
        etf_symbol (str): The ticker symbol for the ETF.

    Returns:
        pd.DataFrame: A DataFrame containing the ETF holdings, or an empty DataFrame if an error occurs.
    """
    url = f"https://www.alphavantage.co/query"
    params = {
        "function": "ETF_PROFILE",
        "symbol": etf_symbol,
        "apikey": api_key
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        if "holdings" in data:
            return pd.DataFrame(data["holdings"])
        elif "Note" in data:
            print(f"API Note: {data['Note']}")
        elif "Error Message" in data:
            print(f"API Error: {data['Error Message']}")
        else:
            print("Unexpected API response format.")

    except requests.exceptions.RequestException as e:
        print(f"HTTP Request failed: {e}")
    except ValueError as e:
        print(f"Error decoding JSON: {e}")

    # Return an empty DataFrame if any error occurs
    return pd.DataFrame()


In [ ]:
# Get the ETF holdings for SPY
if __name__ == "__main__":
    etf_symbol = "SPY"
    holdings = get_etf_holdings(api_key, etf_symbol)

    if not holdings.empty:
        print(holdings.head())
    else:
        print("No holdings data retrieved.")